In [1]:
!python -V

Python 3.7.10


In [2]:
%%capture
!pip install pytorch-lightning==1.5.3 torchmetrics==0.6.0 pycocotools

In [4]:
from pycocotools.coco import COCO
import os
import cv2
import copy

In [5]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [6]:
import ast
import math
import multiprocessing as mp
from pathlib import Path


import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import torchmetrics
import torchvision
import wandb
from PIL import Image
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger
from torchmetrics.metric import Metric
from torchvision.datasets import VisionDataset
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data.dataloader import DataLoader

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
# Please, create u secret key in Add-ons for wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()


wandb_api = user_secrets.get_secret("wandb_api") 

wandb.login(key=wandb_api)

In [9]:
def collate_fn(batch):
    return tuple(zip(*batch))


def get_n_classes(path_):
    train_dataset = COCO_Dataset(path_, split='train')
    return len(train_dataset.coco.cats.keys())


class COCO_Dataset(VisionDataset):

    def __init__(self, root, split='train', transform=None, target_transform=None, transforms=None):
        super().__init__(root, transforms, transform, target_transform)
        self.split = split
        self.coco = COCO(os.path.join(root, split, "_annotations.coco.json"))
        self.ids = list(sorted(self.coco.imgs.keys()))
        self.ids = [id for id in self.ids if (len(self._load_target(id)) > 0)]

    def _load_image(self, id: int):
        path = self.coco.loadImgs(id)[0]['file_name']
        image = cv2.imread(os.path.join(self.root, self.split, path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image

    def _load_target(self, id):
        return self.coco.loadAnns(self.coco.getAnnIds(id))

    def __getitem__(self, index):
        id = self.ids[index]
        image = self._load_image(id)
        target = self._load_target(id)
        target = copy.deepcopy(self._load_target(id))

        boxes = [t['bbox'] + [t['category_id']] for t in target]  # required annotation format for albumentations
        if self.transforms is not None:
            transformed = self.transforms(image=image, bboxes=boxes)

        image = transformed['image']
        boxes = transformed['bboxes']

        new_boxes = []
        for box in boxes:
            xmin = box[0]
            xmax = xmin + box[2]
            ymin = box[1]
            ymax = ymin + box[3]
            new_boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.tensor(new_boxes, dtype=torch.float32)

        targ = {}
        targ['boxes'] = boxes
        targ['labels'] = torch.tensor([t['category_id'] for t in target], dtype=torch.int64)
        targ['image_id'] = torch.tensor([t['image_id'] for t in target])
        targ['area'] = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        targ['iscrowd'] = torch.tensor([t['iscrowd'] for t in target], dtype=torch.int64)
        return image.div(255), targ

    def __len__(self):
        return len(self.ids)

In [10]:
def get_transforms(train=False):
    if train:
        transform = A.Compose([
            A.Resize(600, 600),
            A.HorizontalFlip(p=0.3),
            A.VerticalFlip(p=0.3),
            A.RandomBrightnessContrast(p=0.1),
            A.ColorJitter(p=0.1),
            # A.Normalize(mean=[0.4784, 0.4453, 0.3952], std=[0.2655, 0.2599, 0.2674]),
            ToTensorV2()
        ], bbox_params=A.BboxParams(format='coco'))
    else:
        transform = A.Compose([
            A.Resize(600, 600),
            # A.Normalize(mean=[0.4784, 0.4453, 0.3952], std=[0.2655, 0.2599, 0.2674]),
            ToTensorV2()
        ], bbox_params=A.BboxParams(format='coco'))
    return transform


class COCO_DataModule(pl.LightningDataModule):

    def __init__(self, dataset_path, batch_size, num_workers):
        super().__init__()

        self.save_hyperparameters()

        self.dataset_path = dataset_path
        self.batch_size = batch_size
        self.num_workers = num_workers

    def setup(self, stage=None):
        train_dataset = COCO_Dataset(self.dataset_path, split='train', transforms=get_transforms(True))
        val_dataset = COCO_Dataset(self.dataset_path, split='valid', transforms=get_transforms(True))

        self.train_dataset, self.val_dataset = train_dataset, val_dataset

    def train_dataloader(self):
        return self._dataloader(self.train_dataset, shuffle=True)

    def val_dataloader(self):
        return self._dataloader(self.val_dataset)

    def _dataloader(self, dataset, shuffle=False):
        return DataLoader(
            dataset,
            batch_size=self.batch_size,
            shuffle=shuffle,
            num_workers=self.num_workers,
            collate_fn=collate_fn,
            pin_memory=True,
            drop_last=True,
        )


def collate_fn(batch):
    return tuple(zip(*batch))

In [19]:
from torchvision.models.detection.faster_rcnn import FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

In [12]:

class FRCNNObjectDetector(FasterRCNN):
    def __init__(self, pretrained_weights_path=None, num_classes=8, **kwargs):
        if pretrained_weights_path is None:
            backbone = resnet_fpn_backbone('resnet50', True)
            super(FRCNNObjectDetector, self).__init__(backbone, num_classes, **kwargs)
        else:
            backbone = resnet_fpn_backbone('resnet50', False)
            super(FRCNNObjectDetector, self).__init__(backbone, num_classes, **kwargs)
            self.load_state_dict(torch.load(pretrained_weights_path))

In [13]:

class COCO_Module(pl.LightningModule):

    def __init__(self, pretrained_weights_path=None, num_classes=3):
        super().__init__()

        self.model = self._create_model(pretrained_weights_path, num_classes)

        self.val_map = torchmetrics.MAP()
        self.val_f2 = F2()

    def _create_model(self, pretrained_weights_path, num_classes):
        return FRCNNObjectDetector(pretrained_weights_path, num_classes)

    def forward(self, image):
        self.model.eval()
        output = self.model(image)

        return output

    def training_step(self, batch, batch_idx):
        image, target = batch
        loss_dict = self.model(image, target)
        losses = sum(loss for loss in loss_dict.values())

        batch_size = len(batch[0])
        self.log_dict(loss_dict, batch_size=batch_size)
        self.log("train_loss", losses, batch_size=batch_size)

        return losses

    def validation_step(self, batch, batch_idx):
        image, target = batch
        output = self.model(image)

        val_map = self.val_map(output, target)
        val_f2 = self.val_f2(output, target)

        self.log("val_map", val_map["map"])
        self.log("val_f2", val_f2)

    def configure_optimizers(self):
        params = [p for p in self.model.parameters() if p.requires_grad]
        optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)

        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

        return [optimizer], [lr_scheduler]

In [14]:
def f_beta(tp, fp, fn, beta=2):
    return (1 + beta ** 2) * tp / ((1 + beta ** 2) * tp + beta ** 2 * fn + fp)


In [20]:


class F2(Metric):
    def __init__(
            self,
            compute_on_step=True,
            dist_sync_on_step=False,
            process_group=None,
            dist_sync_fn=None,
    ) -> None:
        super().__init__(
            compute_on_step=compute_on_step,
            dist_sync_on_step=dist_sync_on_step,
            process_group=process_group,
            dist_sync_fn=dist_sync_fn,
        )

        self.add_state("detection_boxes", default=[], dist_reduce_fx=None)
        self.add_state("detection_scores", default=[], dist_reduce_fx=None)
        self.add_state("groundtruth_boxes", default=[], dist_reduce_fx=None)

    def update(self, preds, target):
        for item in preds:
            self.detection_boxes.append(
                torchvision.ops.box_convert(item["boxes"], in_fmt="xywh", out_fmt="xyxy")
                if len(item["boxes"]) > 0
                else item["boxes"]
            )
            self.detection_scores.append(item["scores"])

        for item in target:
            self.groundtruth_boxes.append(
                torchvision.ops.box_convert(item["boxes"], in_fmt="xywh", out_fmt="xyxy")
                if len(item["boxes"]) > 0
                else item["boxes"]
            )

    def compute(self):
        tps, fps, fns = 0, 0, 0
        for gt_boxes, pred_boxes, pred_scores in zip(
                self.groundtruth_boxes, self.detection_boxes, self.detection_scores
        ):
            tp, fp, fn = self._compute_stat_scores(gt_boxes, pred_boxes, pred_scores)
            tps += tp
            fps += fp
            fns += fn

        return f_beta(tps, fps, fns, beta=2)

    def _compute_stat_scores(self, gt_boxes, pred_boxes, pred_scores):
        if len(gt_boxes) == 0 and len(pred_boxes) == 0:
            tps, fps, fns = 0, 0, 0
            return tps, fps, fns

        elif len(gt_boxes) == 0:
            tps, fps, fns = 0, len(pred_boxes), 0
            return tps, fps, fns

        elif len(pred_boxes) == 0:
            tps, fps, fns = 0, 0, len(gt_boxes)
            return tps, fps, fns

        _, indices = torch.sort(pred_scores, descending=True)
        pred_boxes = pred_boxes[indices]

        tps, fps, fns = 0, 0, 0
        for iou_th in np.arange(0.3, 0.85, 0.05):
            tp, fp, fn = self._compute_stat_scores_at_iou_th(gt_boxes, pred_boxes, iou_th)
            tps += tp
            fps += fp
            fns += fn

        return tps, fps, fns

    def _compute_stat_scores_at_iou_th(self, gt_boxes, pred_boxes, iou_th):
        gt_boxes = gt_boxes.clone()
        pred_boxes = pred_boxes.clone()

        tp = 0
        fp = 0
        for k, pred_bbox in enumerate(pred_boxes):
            ious = torchvision.ops.box_iou(gt_boxes, pred_bbox[None, ...])

            max_iou = ious.max()
            if max_iou > iou_th:
                tp += 1

                argmax_iou = ious.argmax()
                gt_boxes = torch.cat([gt_boxes[0:argmax_iou], gt_boxes[argmax_iou + 1:]])
            else:
                fp += 1
            if len(gt_boxes) == 0:
                fp += len(pred_boxes) - (k + 1)
                break

        fn = len(gt_boxes)

        return tp, fp, fn

In [16]:

cfg = {
    'dataset_path': "/kaggle/input/oxford-fixed",
'batch_size': 4,
'epoch_num': 10,

'checkpoint_pl': "",

'wandb_project_name': "ai_cloud_demo",
'wandb_run_name': "fastrcnn_Oxford_10_ep_pretrained",

'fast_dev_run': False,

'checkpoint_pl_output': "./pl_checkpointOxford_pretrained/",
'model_state_dict_output': "./Oxford_state_dict/",

'gpus_num': 1,
}

In [21]:

!mkdir Oxford_state_dict

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

wandb: wandb version 0.13.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade

CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
loss_box_reg,█▆██▇▃▅▃▅▄▇▃█▃▄▄▄▅▃▂▄▆▃▃▄▄▂▂▁▆▄▁▂▂▆▂▄▁▄▃
loss_classifier,█▆▆█▅▄▃▃▄▃▃▂▄▂▃▃▂▃▃▂▂▃▂▃▂▂▂▂▁▄▁▂▁▂▄▁▂▁▂▂
loss_objectness,█▄▃▆▂▃▂▁▂▃▂▁█▁▂▁▂▁▁▁▁▂▂▂▁▁▂▂▂▂▁▁▁▁▁▁▂▁▁▁
loss_rpn_box_reg,▅▃▄▆▅▃▄▃▄▃▂▆█▂▃▂▂▅▁▂▃▂▁▂▂▂▂▂▂▄▁▄▁▂▂▁▁▂▁▂
train_loss,█▅▆█▆▃▄▃▄▃▄▂▆▂▃▃▃▄▂▂▃▄▂▃▂▂▂▂▁▄▂▂▁▂▄▁▃▁▃▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_f2,▁▃▄▅▆▅▆▆▆█
val_map,▁▄▅▆▇▆▇▇██
epoch,9
loss_box_reg,0.05128


In [ ]:
def train():
    NUM_WORKERS = mp.cpu_count()
    data_module = COCO_DataModule(cfg['dataset_path'], cfg['batch_size'], NUM_WORKERS)
    if cfg['checkpoint_pl']:
        model = COCO_Module.load_from_checkpoint(cfg['checkpoint_pl'])
    else:
        model = COCO_Module(num_classes=get_n_classes(cfg['dataset_path']))
    wandb.init(project=cfg['wandb_project_name'], name=cfg['wandb_run_name'])
    trainer = pl.Trainer(
        default_root_dir=cfg['checkpoint_pl_output'],
        fast_dev_run=cfg['fast_dev_run'],  # FAST_DEV_RUN,
        gpus=cfg['gpus_num'],
        logger=WandbLogger(project=cfg['wandb_project_name'], log_model=True, mode='online'),
        max_epochs=cfg['epoch_num'],
        precision=16 if cfg['gpus_num'] else 32,
        log_every_n_steps=1
    )
    trainer.fit(model, data_module)
    
    torch.save(
    model.model.state_dict(),
    cfg['model_state_dict_output'] + f"{cfg['wandb_run_name']}_model.pth"
    )
    wandb.finish()
train()

In [ ]:

import shutil
shutil.make_archive('data', 'zip', '/kaggle/working/')